In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install numpy 
!pip install pandas
!pip install sklearn
!pip install torch 
!pip install sklearn
!pip install seaborn
!pip install matplotlib
!pip install json
!pip install fast_ml

     |████████████████████████████████| 4.2 MB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 64.0 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 596 kB 63.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
     |████████████████████████████████| 42 kB 522 kB/s 


In [ ]:
import numpy as np #Für Matrizen
import pandas as pd #Für Dataframes
import re
from typing import List, Any, Set, Dict, Tuple, Optional
from ast import literal_eval
import json # load json module

import torch
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset
from transformers import BertTokenizer, AutoTokenizer
from transformers import BertForSequenceClassification
from fast_ml.model_development import train_valid_test_split

In [ ]:
dict_name = "l_dict_m2"
json_dir = f'/content/drive/MyDrive/{dict_name}.json'

In [ ]:
with open(json_dir) as jsonFile:
    labels_Input = json.load(jsonFile)
    jsonFile.close()
print(max(labels_Input), min(labels_Input), len(labels_Input))

9999005 -1 89856


In [ ]:
labels_Output = {}
for k, v in labels_Input.items():
    labels_Output[v["index"]] = {
            "name": v["name"],
            "synonym": v["synonym"],
            "code": k,
            "index": v["index"]
            } 
print(len(labels_Output))

89856


**Trainingsdaten vorbereiten**

In [ ]:
excel_path = '/content/drive/MyDrive/train_data.xlsx'
sheet_name = "labeled_dataset2"

In [ ]:
df = pd.read_excel(excel_path, sheet_name, index_col=None, dtype={'Label': np.int64})
df = df.astype({'Label': str})
print(df)
print(len(df))

In [ ]:
df['Label'].value_counts()

In [ ]:
label_code = []
code_fails = []
for index, row in df.iterrows():
    if row["Label"] in labels_Input:
        label_code.append(labels_Input[row["Label"]]["index"])
    else:
        label_code.append(labels_Input["-1"]["index"])

df['label_code'] = label_code
print(df)

In [ ]:
labels_Output = {}
for k, v in labels_Input.items():
    labels_Output[v["index"]] = {
      "name": v["name"],
      "synonym": v["synonym"],
      "code": k,
      "index": v["index"]
  } 

print(min(labels_Output), max(labels_Output), len(labels_Output))

In [ ]:
deep_copy = df.copy(deep=True)
df = df.append(deep_copy)
df['Label'].value_counts()

In [ ]:
df['Label'].value_counts()

In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df, target = 'label_code', 
                                                                            train_size=0.8, valid_size=0.1, test_size=0.1)



df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train.index.values, 'data_type'] = 'train'
df.loc[X_valid.index.values, 'data_type'] = 'val'
df.loc[X_test.index.values, 'data_type'] = 'test'

df.groupby(['Label', 'label_code', 'data_type']).count()

**Vorbereitete Trainingsdaten laden**

In [ ]:
excel_path = '/content/drive/MyDrive/train_ready_data.xlsx'
sheet_name = "train_ready_dataset"

In [ ]:
df = pd.read_excel(excel_path, sheet_name, index_col=None, dtype={'Label': np.int64})
df = df.astype({'Label': str})
print(df)
print(len(df))

                                                     Text      Label  \
0               Excision of lesion of patella (procedure)     104001   
1                           Excision of lesion of patella     104001   
2           Local excision of lesion or tissue of patella     104001   
3       Structure of posterior carpal region (body str...     106004   
4                                 Posterior carpal region     106004   
...                                                   ...        ...   
241398  intestinal continuity according to Hartmann\n1...  405815000   
241399  intestinal continuity according to Hartmann\n1...   86174004   
241400  Stenosis of jejunum, Cerebrovascular accident ...         -1   
241401  Stenosis of jejunum, Cerebrovascular accident ...         -1   
241402  countable procedures\n(Thoracic surgery, Urolo...    1904001   

        label_code data_type  
0                0     train  
1                0     train  
2                0       val  
3          

**Training vorbereiten**

In [ ]:
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.2')

print("Encode Train Data.......")
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)
print("Encode Validation Data.......")
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)
print("Encode Test Data.......")
encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type=='test'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label_code.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label_code.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df[df.data_type=='test'].label_code.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Encode Train Data.......
Encode Validation Data.......
Encode Test Data.......


In [ ]:
model = BertForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.2',
                                                      num_labels=len(labels_Input),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16 

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import Adam

optimizer = Adam(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs =  8

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
import random
import torch
import numpy as np

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        #print(loss)
        logits = outputs[1]
        #print(logits)
        loss_val_total += loss.item()
        #print(loss_val_total)

        logits = logits.detach().cpu().numpy()
        #print(logits)
        label_ids = inputs['labels'].cpu().numpy()
        #print(label_ids)

        #print(label_ids)
        predictions.append(logits)
        true_vals.append(label_ids)


    loss_val_avg = loss_val_total/len(dataloader_val) 
    #print(predictions)
    #print(true_vals)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    #print(len(predictions))
    #print(len(true_vals))

            
    return loss_val_avg, predictions, true_vals

In [ ]:
import random
import torch
import numpy as np


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


def test(dataloader_test):

    model.eval()
    
    predictions, true_vals = [], []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)     

        logits = outputs[1]

        logits = logits.detach().cpu().numpy()

        label_ids = inputs['labels'].cpu().numpy()

        predictions.append(logits)
        true_vals.append(label_ids)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return predictions, true_vals

**Training durchführen**

In [ ]:
tl = []
vl = []
fo = []
# set initial loss to infinite
best_valid_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    tl.append(loss_train_avg)
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(true_vals, predictions)
    tqdm.write(f'Validation loss: {val_loss}')
    vl.append(val_loss)
    
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/saved_weights_M2_v1.model')  
        
    tqdm.write(f'F1 Score (Weighted): {val_f1}')
    fo.append(val_f1)

torch.save(model.state_dict(), '/content/drive/MyDrive/saved_weights_M2_v1.model')  
   
for i in tl:
  print(i)

for i in vl:
  print(i)

for i in fo:
  print(i)

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/11304 [00:00<?, ?it/s]


Epoch 1
Training loss: 5.7639297583448235
Validation loss: 8.674346309111863
F1 Score (Weighted): 0.18790665274644253


Epoch 2:   0%|          | 0/11304 [00:00<?, ?it/s]


Epoch 2
Training loss: 5.738341946510753
Validation loss: 8.564772912055727
F1 Score (Weighted): 0.19547504022040604


...


Epoch 6
Training loss: 5.6149464887418565
Validation loss: 8.31975709193598
F1 Score (Weighted): 0.20866855360204664


Epoch 7:   0%|          | 0/11304 [00:00<?, ?it/s]


Epoch 7
Training loss: 5.682670274146972
Validation loss: 8.298572687371067
F1 Score (Weighted): 0.20894767830956412


Epoch 8:   0%|          | 0/11304 [00:00<?, ?it/s]


Epoch 8
Training loss: 5.804195775748649
Validation loss: 8.291488309890505
F1 Score (Weighted): 0.20965620680002167
5.7639297583448235
5.738341946510753
5.642967034191926
5.591828878771221
5.584378748330678
5.6149464887418565
5.682670274146972
5.804195775748649
8.674346309111863
8.564772912055727
8.472760139571296
8.405504209276229
8.355412039554938
8.31975709193598
8.298572687371067
8.291488309890505
0.18790665274644253
0.19547504022040604
0.20000789905511432
0.20333556051385446
0.2066619512967717
0.20866855360204664
0.20894767830956412
0.20965620680002167


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/saved_weights_M2_v1.model')  

In [ ]:
print(tl)
print(vl)
print(fo)

In [ ]:
for i in tl:
  print(i)

In [ ]:
for i in vl:
  print(i)

In [ ]:
for i in fo:
  print(i)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(tl, vl)
plt.xlabel("Epochen")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(fo)
plt.xlabel("Epochen")
plt.ylabel("Val F1")
plt.show()

**Testen**

In [ ]:
bert_base_model = "dmis-lab/biobert-base-cased-v1.2"
training_state = '/content/drive/MyDrive/saved_weights_M2_v1.model'

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

model = BertForSequenceClassification.from_pretrained(bert_base_model,
                                                      num_labels=len(labels_Input),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
print("BFSC as Model loaded....")

model.load_state_dict(torch.load(training_state, map_location=device))
print("state_dict loaded.....")

#tokenizer = BertTokenizer.from_pretrained(bert_base_model, 
                                          #output_hidden_states=False,
                                          #map_location=device)
#print("Tokenizer loaded.....")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

BFSC as Model loaded....
state_dict loaded.....


In [ ]:
predictions, true_vals = test(dataloader_test)

In [ ]:
val_f1 = f1_score_func(true_vals, predictions)
tqdm.write(f'F1 Score (Weighted): {val_f1}')

F1 Score (Weighted): 0.20737828337066


In [ ]:
preds = np.argmax(predictions, axis = 1)
print(classification_report(true_vals, preds))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         1
          23       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         0
          30       1.00      1.00      1.00         1
          31       1.00      1.00      1.00         1
          32       1.00      1.00      1.00         1
          35       0.00      0.00      0.00         3
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         1
          40       0.00    

In [ ]:
sentence = "Comprehensive care of lower urinary tract infections"
model.eval()

input = tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors = "pt")
output = model(**input)
logits = output.logits
soft = torch.nn.Softmax(dim=0)
soft_tensor = soft(logits[0])
probs = soft_tensor.detach().numpy()
#print(probs)
probabilities = {}
for k, v in labels_Input.items():
    probabilities[k] = probs[v["index"]]
print(probabilities)
print("-----------------------------------------------------------------------------------------")
